In [1]:
import os
os.environ['OMP_NUM_THREADS'] = '1'

import sys
sys.path.append(os.path.join(os.getcwd(), *tuple(['..'])))
import argparse

from typing import Callable, Dict, List, Optional, Set
from collections import OrderedDict
import pdb
import numpy as np
import torch
from model_mrcnn import _default_mrcnn_config, build_default
from features import build_features
from features import transforms as T
from utils.engine import evaluate
import torchvision
from utils.helper_functions import evaluate_metrics, get_outputs
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import multilabel_confusion_matrix

In [2]:
#python -m ipykernel install --user --name=kfold_amy_plaque1

In [3]:
test_config = dict(
    batch_size = 8,
    num_classes=4,
    device_id =0
)

In [4]:
#model_name = os.path.join("/gladstone/finkbeiner/steve//work/data/npsad_data/vivek/models/stellar-pyramid-31_mrcnn_model_100.pth")
#model_name = os.path.join("/gladstone/finkbeiner/steve//work/data/npsad_data/vivek/models/lyric-puddle-40_mrcnn_model_80.pth")
#model_name = os.path.join("/gladstone/finkbeiner/steve//work/data/npsad_data/vivek/models/visionary-salad-42_mrcnn_model_80.pth")
#model_name = os.path.join("/gladstone/finkbeiner/steve//work/data/npsad_data/vivek/models/devout-oath-43_mrcnn_model_80.pth")
#model_name = os.path.join("/gladstone/finkbeiner/steve//work/data/npsad_data/vivek/models/devout-oath-43_mrcnn_model_30.pth")
#model_name = "/workspace/Projects/Amyb_plaque_detection/models/different-galaxy-44_mrcnn_model_70.pth"
#model_name = "/workspace/Projects/Amyb_plaque_detection/models/icy-yogurt-62_mrcnn_model_25.pth"
model_name = "/workspace/Projects/Amyb_plaque_detection/models/dulcet-pond-63_mrcnn_model_40.pth"

In [5]:
dataset_test_location = '/workspace/Projects/Amyb_plaque_detection/Datasets/val'
test_patient_ids = os.listdir(dataset_test_location)

In [6]:
test_patient_ids.remove('.DS_Store')

In [7]:
collate_fn = lambda _: tuple(zip(*_)) # one-liner, no need to import

In [8]:
model_config = _default_mrcnn_config(num_classes=1+test_config["num_classes"]).config
model = build_default(model_config, im_size=1024)
model.load_state_dict(torch.load(model_name))
device = torch.device('cpu')
device = torch.device('cuda', test_config['device_id'])
model = model.to(device) 
model.eval()


all_patients_val_metrics = pd.DataFrame()
for t in range(len(test_patient_ids)):
    if len(os.listdir(os.path.join(dataset_test_location,test_patient_ids[t],"images")))==0:
        continue
    test_ds = build_features.AmyBDataset(os.path.join(dataset_test_location,test_patient_ids[t]),T.Compose([T.ToTensor()]))
    test_loader = torch.utils.data.DataLoader(test_ds, batch_size=test_config['batch_size'], shuffle=False, num_workers=4, collate_fn=collate_fn)

    final_metric_df =pd.DataFrame()
    for threshold in [0.25,0.5,0.75,0.9,0.95]:
        f1_mean_list = []
        labels_matched_list = []
        actual_label_list = []
        pred_label_list = []
        pred_score_list = []
        for i, (images, targets) in enumerate(test_loader):
            images = [image.to(device) for image in images]
            targets = [dict([(k, v.to(device)) for k, v in target.items()]) for target in targets]
            outputs = model.forward(images, targets)
            masks, labels, scores = get_outputs(outputs, threshold)
            f1_mean, labels_matched, actual_labels,pred_labels, pred_scores =  evaluate_metrics(targets, masks, labels, scores)
            if len(f1_mean)>0 or len(labels_matched)>0:
                #print(" Validation f1 mean score:", f1_mean, " perc labels matched", labels_matched)
                f1_mean_list.extend(f1_mean)
                labels_matched_list.extend(labels_matched)
                actual_label_list.extend(actual_labels)
                pred_label_list.extend(pred_labels)
                pred_score_list.extend(pred_scores)
        metric_df = pd.DataFrame({"f1_score":f1_mean_list,"labels_matched":labels_matched_list,"actual_labels":actual_label_list,
                                  "pred_labels":pred_label_list,"pred_scores":pred_score_list})
        metric_df["threshold"] = threshold
        if  len(final_metric_df)>0:
            final_metric_df=pd.concat([final_metric_df,metric_df],axis=0, ignore_index=True)
        else:
            final_metric_df=metric_df
    final_metric_df["patient_id"]=test_patient_ids[t]
    if len(all_patients_val_metrics)>0:
        all_patients_val_metrics=pd.concat([final_metric_df,all_patients_val_metrics],axis=0, ignore_index=True)
    else:
        all_patients_val_metrics=final_metric_df


/root/miniconda3/envs/kfold_amy_plaque1/lib/python3.9/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'backbone_name' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/root/miniconda3/envs/kfold_amy_plaque1/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/miniconda3/envs/kfold_amy_plaque1/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/workspace/Projects/Amyb_plaque_detectio

In [9]:
all_patients_val_metrics

,f1_score,labels_matched,actual_labels,pred_labels,pred_scores,threshold,patient_id
0,0.938408,1,2,2,0.984658,0.25,XE07-047_1_AmyB_1
1,0.589191,0,2,3,0.689315,0.25,XE07-047_1_AmyB_1
2,0.595951,0,2,1,0.337996,0.25,XE07-047_1_AmyB_1
3,NaN,0,2,1,0.905074,0.25,XE07-047_1_AmyB_1
4,0.061603,0,2,1,0.720439,0.25,XE07-047_1_AmyB_1
...,...,...,...,...,...,...,...
3711,NaN,0,3,1,0.977445,0.95,XE19-028_1_AmyB_1
3712,NaN,0,3,1,0.970916,0.95,XE19-028_1_AmyB_1
3713,0.472426,1,1,1,0.956956,0.95,XE19-028_1_AmyB_1
3714,0.938470,1,1,1,0.956956,0.95,XE19-028_1_AmyB_1


In [10]:
for threshold in [0.25,0.5]:
    print("------------threshold--------------",threshold)
    all_patients_val_metrics1 = all_patients_val_metrics[~all_patients_val_metrics["f1_score"].isna()]
    all_patients_val_metrics2 = all_patients_val_metrics1[all_patients_val_metrics1["threshold"]==threshold]
    precision, recall,fbeta_score, support = precision_recall_fscore_support(all_patients_val_metrics2["actual_labels"], all_patients_val_metrics2["pred_labels"])
    print("Precision :",precision)
    print("recall :",recall)
    print("support :", support)
    print(multilabel_confusion_matrix(all_patients_val_metrics2["actual_labels"], all_patients_val_metrics2["pred_labels"], labels = [1.0,2.0,3.0,4.0]))


------------threshold-------------- 0.25
Precision : [0.38356164 0.50997151 0.15625    0.51219512]
recall : [0.375      0.50280899 0.24271845 0.3255814 ]
support : [224 356 103 129]
[[[453 135]
  [140  84]]

 [[284 172]
  [177 179]]

 [[574 135]
  [ 78  25]]

 [[643  40]
  [ 87  42]]]
------------threshold-------------- 0.5
Precision : [0.53846154 0.55357143 0.18666667 0.50943396]
recall : [0.45528455 0.53142857 0.31111111 0.47368421]
support : [123 175  45  57]
[[[229  48]
  [ 67  56]]

 [[150  75]
  [ 82  93]]

 [[294  61]
  [ 31  14]]

 [[317  26]
  [ 30  27]]]


In [ ]:
all_patients_val_metrics1 = all_patients_val_metrics[~all_patients_val_metrics["f1_score"].isna()]
all_patients_val_metrics2 = all_patients_val_metrics1[all_patients_val_metrics1["threshold"]==0.5]

act_labels = all_patients_val_metrics2["actual_labels"].values

y_test = np.zeros((act_labels.size, act_labels.max()))
y_test[np.arange(act_labels.size), act_labels] = 1



    

for i in range(n_classes):
    precision[i], recall[i], _ = precision_recall_curve(y_test[:, i],
                                                        y_score[:, i])
    plt.plot(recall[i], precision[i], lw=2, label='class {}'.format(i))



In [36]:
for threshold in [0.25,0.5]:
    print("------------threshold--------------",threshold)
    all_patients_val_metrics1 = all_patients_val_metrics[~all_patients_val_metrics["f1_score"].isna()]
    all_patients_val_metrics2 = all_patients_val_metrics1[all_patients_val_metrics1["threshold"]==threshold]
    precision, recall,fbeta_score, support = precision_recall_fscore_support(all_patients_val_metrics2["actual_labels"], all_patients_val_metrics2["pred_labels"])
    print("Precision :",precision)
    print("recall :",recall)
    print("support :", support)
    print(multilabel_confusion_matrix(all_patients_val_metrics2["actual_labels"], all_patients_val_metrics2["pred_labels"], labels = [1.0,2.0,3.0,4.0]))


------------threshold-------------- 0.25
Precision : [0.36734694 0.55349794 0.1377551  0.44791667]
recall : [0.38028169 0.55010225 0.23076923 0.23626374]
support : [284 489 117 182]
[[[602 186]
  [176 108]]

 [[366 217]
  [220 269]]

 [[786 169]
  [ 90  27]]

 [[837  53]
  [139  43]]]
------------threshold-------------- 0.5
Precision : [0.42857143 0.58796296 0.         0.21428571]
recall : [0.61538462 0.65463918 0.         0.0625    ]
support : [117 194  39  48]
[[[185  96]
  [ 45  72]]

 [[115  89]
  [ 67 127]]

 [[359   0]
  [ 39   0]]

 [[339  11]
  [ 45   3]]]


/root/miniconda3/envs/kfold_amy_plaque1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
all_patients_val_metrics2["actual_labels"].unique()

array([2.])

In [23]:
multilabel_confusion_matrix(all_patients_val_metrics2["actual_labels"], all_patients_val_metrics2["pred_labels"], labels = [

In [30]:
all_patients_val_metrics2

,f1_score,labels_matched,actual_labels,pred_labels,pred_scores,threshold,patient_id
0,0.924727,1.0,2.0,2.0,0.915549,0.25,XE07-047_1_AmyB_1
2,0.623869,0.0,2.0,1.0,0.626739,0.25,XE07-047_1_AmyB_1
3,0.619643,0.0,2.0,3.0,0.352768,0.25,XE07-047_1_AmyB_1
6,0.074731,0.0,2.0,1.0,0.749540,0.25,XE07-047_1_AmyB_1
7,0.077944,0.0,2.0,3.0,0.372418,0.25,XE07-047_1_AmyB_1
...,...,...,...,...,...,...,...
3425,0.343465,0.0,2.0,1.0,0.738178,0.25,XE19-028_1_AmyB_1
3426,0.350675,0.0,2.0,3.0,0.344608,0.25,XE19-028_1_AmyB_1
3427,0.884177,0.0,2.0,1.0,0.413591,0.25,XE19-028_1_AmyB_1
3428,0.907886,1.0,2.0,2.0,0.343253,0.25,XE19-028_1_AmyB_1


In [ ]:
multilabel_confusion_matrix(y_true, y_pred)

In [24]:
precision, recall,fbeta_score, support = precision_recall_fscore_support(all_patients_val_metrics2["actual_labels"], all_patients_val_metrics2["pred_labels"])

In [25]:
precision

array([0.36734694, 0.55349794, 0.1377551 , 0.44791667])

In [26]:
recall

array([0.38028169, 0.55010225, 0.23076923, 0.23626374])

In [17]:
precision, recall, thresholds = precision_recall_curve(
    all_patients_val_metrics1["actual_labels"], all_patients_val_metrics1["pred_scores"])
    

ValueError: multiclass format is not supported

In [28]:
for pat_id in test_patient_ids:
    print("Patient id : ", pat_id)
    for threshold in [0.25,0.5,0.75,0.9,0.95]:
        df1 = all_patients_val_metrics[(all_patients_val_metrics["threshold"]==threshold) & (~all_patients_val_metrics["f1_score"].isna()) & 
        (all_patients_val_metrics["patient_id"]==pat_id)]
        print("threshold:",threshold )
        print(df1["f1_score"].sum()/len(df1))
        print(df1["labels_matched"].sum()/len(df1))
        precision, recall,fbeta_score, support = precision_recall_fscore_support(df1["actual_labels"], df1["pred_labels"])
        print("Precision :",precision)
        print("recall :",recall)
        print("support :", support)
        print("-----------------------------")
        

Patient id :  XE19-028_1_AmyB_1
threshold: 0.25
0.40385418297506065
0.2876712328767123
Precision : [0.33121019 0.37804878 0.16161616 0.22222222]
recall : [0.44444444 0.20945946 0.25       0.16666667]
support : [117 148  64  36]
-----------------------------
threshold: 0.5
0.4106981860163428
0.34911242603550297
Precision : [0.36082474 0.35483871 0.         0.2       ]
recall : [0.58333333 0.33846154 0.         0.11764706]
support : [60 65 27 17]
-----------------------------
threshold: 0.75
0.359016547912908
0.36363636363636365
Precision : [0.         0.36363636 0.         0.        ]
recall : [0. 1. 0. 0.]
support : [10 12  6  5]
-----------------------------
threshold: 0.9
0.48744423734900394
0.3333333333333333
Precision : [0.         0.33333333 0.         0.        ]
recall : [0. 1. 0. 0.]
support : [2 2 1 1]
-----------------------------
threshold: 0.95
nan
nan
Precision : []
recall : []
support : []
-----------------------------
Patient id :  XE17-004_1_AmyB_1
threshold: 0.25
0.279

/root/miniconda3/envs/kfold_amy_plaque1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/miniconda3/envs/kfold_amy_plaque1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/miniconda3/envs/kfold_amy_plaque1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

In [9]:
for threshold in [0.25,0.5,0.75,0.9,0.95]:
    df1 = final_metric_df[(final_metric_df["threshold"]==threshold) & (~final_metric_df["f1_score"].isna())]
    print("threshold:",threshold )
    print(df1["f1_score"].sum()/len(df1))
    print(df1["labels_matched"].sum()/len(df1))
    print("-----------------------------")

threshold: 0.25
0.5140259231042068
0.49206349206349204
-----------------------------
threshold: 0.5
0.5873500186472632
0.5909090909090909
-----------------------------
threshold: 0.75
0.6211402353714853
0.7027027027027027
-----------------------------
threshold: 0.9
0.6530133832188733
0.7619047619047619
-----------------------------
threshold: 0.95
0.6543345995514597
0.75
-----------------------------


In [15]:
final_metric_df

,f1_score,labels_matched,actual_labels,pred_labels,threshold,patient_id
0,0.945723,1,2,2,0.25,XE07-047_1_AmyB_1
1,NaN,0,2,2,0.25,XE07-047_1_AmyB_1
2,0.623079,0,2,1,0.25,XE07-047_1_AmyB_1
3,0.621790,0,2,3,0.25,XE07-047_1_AmyB_1
4,NaN,0,2,1,0.25,XE07-047_1_AmyB_1
...,...,...,...,...,...,...
1420,NaN,0,1,2,0.95,XE07-047_1_AmyB_1
1421,0.623704,0,1,2,0.95,XE07-047_1_AmyB_1
1422,NaN,0,1,2,0.95,XE07-047_1_AmyB_1
1423,NaN,0,1,2,0.95,XE07-047_1_AmyB_1


In [9]:
for threshold in [0.25,0.5,0.75,0.9,0.95]:
    df1 = final_metric_df[final_metric_df["threshold"]==threshold]
    print(precision_recall_fscore_support(df1["actual_labels"],df1["pred_labels"]))

(array([0.53535354, 0.41772152, 0.08695652]), array([0.42063492, 0.32352941, 0.31578947]), array([0.47111111, 0.36464088, 0.13636364]), array([126, 102,  19]))
(array([0.52631579, 0.41176471, 0.10638298]), array([0.46296296, 0.32941176, 0.29411765]), array([0.49261084, 0.36601307, 0.15625   ]), array([108,  85,  17]))
(array([0.52808989, 0.42857143, 0.1       ]), array([0.53409091, 0.38571429, 0.14285714]), array([0.53107345, 0.40601504, 0.11764706]), array([88, 70, 14]))
(array([0.546875  , 0.42857143, 0.        ]), array([0.57377049, 0.45652174, 0.        ]), array([0.56      , 0.44210526, 0.        ]), array([61, 46, 10]))
(array([0.54545455, 0.43902439, 0.        ]), array([0.57142857, 0.51428571, 0.        ]), array([0.55813953, 0.47368421, 0.        ]), array([42, 35,  8]))


/home/mahirwar/miniconda3/envs/kfold_amy_plaque1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
for threshold in [0.25,0.5,0.75,0.9,0.95]:
    df1 = final_metric_df[(final_metric_df["threshold"]==threshold) & (~final_metric_df["f1_score"].isna())]
    print("threshold:",threshold )
    print(df1["f1_score"].sum()/len(df1))
    print(df1["labels_matched"].sum()/len(df1))
    print("-----------------------------")

threshold: 0.25
0.5440338361800064
0.4943820224719101
-----------------------------
threshold: 0.5
0.5481984925637111
0.5333333333333333
-----------------------------
threshold: 0.75
0.5447650040046997
0.5901639344262295
-----------------------------
threshold: 0.9
0.5629299524388324
0.6341463414634146
-----------------------------
threshold: 0.95
0.553742570799755
0.6666666666666666
-----------------------------


: 

In [10]:
for threshold in [0.25,0.5,0.75,0.9,0.95]:
    df1 = final_metric_df[final_metric_df["threshold"]==threshold]
    print(precision_recall_fscore_support(df1["actual_labels"],df1["pred_labels"]))

(array([0.53191489, 0.44444444, 0.2       , 0.        ]), array([0.64935065, 0.19047619, 0.41666667, 0.        ]), array([0.58479532, 0.26666667, 0.27027027, 0.        ]), array([77, 63, 12,  0]))
(array([0.53731343, 0.45454545, 0.25      ]), array([0.8       , 0.13513514, 0.375     ]), array([0.64285714, 0.20833333, 0.3       ]), array([45, 37,  8]))
(array([0.52830189, 0.        , 0.33333333]), array([0.96551724, 0.        , 0.2       ]), array([0.68292683, 0.        , 0.25      ]), array([29, 22,  5]))
(array([0.53191489, 0.        , 0.33333333]), array([0.96153846, 0.        , 0.2       ]), array([0.68493151, 0.        , 0.25      ]), array([26, 19,  5]))
(array([0.51515152, 0.        , 0.33333333]), array([0.94444444, 0.        , 0.25      ]), array([0.66666667, 0.        , 0.28571429]), array([18, 14,  4]))


/home/mahirwar/miniconda3/envs/kfold_amy_plaque1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mahirwar/miniconda3/envs/kfold_amy_plaque1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mahirwar/miniconda3/envs/kfold_amy_plaque1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(aver

In [14]:
final_metric_df["actual_labels"].unique()

array([2, 1, 3])

In [15]:
final_metric_df["pred_labels"].unique()

array([1, 3, 2, 4])

: 

In [11]:
for threshold in [0.25,0.5,0.75,0.9,0.95]:
    df1 = final_metric_df[(final_metric_df["threshold"]==threshold) & (~final_metric_df["f1_score"].isna())]
    print("threshold:",threshold )
    print(df1["f1_score"].sum()/len(df1))
    print(df1["labels_matched"].sum()/len(df1))
    print("-----------------------------")

threshold: 0.25
0.5094946067334855
0.5
-----------------------------
threshold: 0.5
0.6445666446608866
0.6875
-----------------------------
threshold: 0.75
0.6454421695947034
0.7272727272727273
-----------------------------
threshold: 0.9
0.6890332190196364
0.7368421052631579
-----------------------------
threshold: 0.95
0.6881173185693391
0.7692307692307693
-----------------------------


In [21]:
for threshold in [0.25,0.5,0.75,0.9,0.95]:
    df1 = final_metric_df[final_metric_df["threshold"]==threshold]
    print(precision_recall_fscore_support(df1["actual_labels"],df1["pred_labels"]))

(array([0.525     , 0.38974359, 0.07575758]), array([0.22764228, 0.56505576, 0.22727273]), array([0.31758034, 0.46130501, 0.11363636]), array([369, 269,  44]))
(array([0.53676471, 0.39449541, 0.        ]), array([0.38020833, 0.61870504, 0.        ]), array([0.44512195, 0.48179272, 0.        ]), array([192, 139,  23]))
(array([0.42857143, 0.41441441, 0.        ]), array([0.05084746, 0.93877551, 0.        ]), array([0.09090909, 0.575     , 0.        ]), array([59, 49, 10]))
(array([0.        , 0.40350877, 0.        ]), array([0., 1., 0.]), array([0.   , 0.575, 0.   ]), array([29, 23,  5]))
(array([0.        , 0.35294118]), array([0., 1.]), array([0.        , 0.52173913]), array([11,  6]))


/home/mahirwar/miniconda3/envs/kfold_amy_plaque1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mahirwar/miniconda3/envs/kfold_amy_plaque1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mahirwar/miniconda3/envs/kfold_amy_plaque1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_

In [27]:
for threshold in [0.25,0.5,0.75,0.9,0.95]:
    df1 = final_metric_df[(final_metric_df["threshold"]==threshold) & (~final_metric_df["f1_score"].isna())]
    print("threshold:",threshold )
    print(df1["f1_score"].sum()/len(df1))
    print(df1["labels_matched"].sum()/len(df1))
    print("-----------------------------")


threshold: 0.25
0.4210295890073888
0.4266666666666667
-----------------------------
threshold: 0.5
0.4373702965521511
0.5420560747663551
-----------------------------
threshold: 0.75
0.44116988677374314
0.5757575757575758
-----------------------------
threshold: 0.9
0.4544035777050605
0.5555555555555556
-----------------------------
threshold: 0.95
0.5958336758353802
0.6666666666666666
-----------------------------


: 

In [11]:
metric_df

,f1_score,labels_matched,actual_labels,pred_labels,threshold
0,NaN,0,1,2,0.95
1,NaN,0,1,2,0.95
2,NaN,0,2,2,0.95
3,NaN,0,1,2,0.95
4,0.648812,1,2,2,0.95
5,NaN,0,1,2,0.95
6,NaN,0,2,2,0.95
7,NaN,0,1,2,0.95
8,NaN,0,1,2,0.95
9,NaN,0,2,2,0.95


In [15]:
pd.concat([final_metric_df,metric_df],axis=0, ignore_index=True)

,f1_score,labels_matched,actual_labels,pred_labels,threshold
0,NaN,0,1,2,0.95
1,NaN,0,1,2,0.95
2,NaN,0,2,2,0.95
3,NaN,0,1,2,0.95
4,0.648812,1,2,2,0.95
5,NaN,0,1,2,0.95
6,NaN,0,2,2,0.95
7,NaN,0,1,2,0.95
8,NaN,0,1,2,0.95
9,NaN,0,2,2,0.95
